---
# 1 Introduccion

Para este segundo ejercicios a eleccion, el concepto de HPC aplicado es volumen de dato, donde en la primera parte graficamos la cantidad de casos, muertes o recuperados en función del tiempo según la entrada que el usuario haya seleccionado. El dataSet utilizado consta de una tabla con 279 filas que corresponden a la cantidad de casos de cada día por cada país y aproximadamente 540 columnas, de las cuales 536 corresponden a las fechas. Este dataSet se actualiza cada día, por lo tanto, la cantidad de columnas aumentará en 1 cada día.


En la segunda parte usamos una biblioteca llamada profet (internamente usa la biblioteca sci learn) la cual nos permite pronosticar la cantidad de casos para los próximos meses. Esto se logra entrenando al algoritmo con nuestro dataset para generar un modelo predictivo y luego se obtiene pronóstico para un periodo dado.


---
# 2 Armado del ambiente



---
## 2.1 Instalacion del modulo fbprophet

In [ ]:
!pip install fbprophet

---
## 2.2 Importacion de modulos y datasets necesarios

In [2]:
%matplotlib inline

# Importando paquetes necesarios
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from fbprophet import Prophet
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from urllib.error import HTTPError

# Seteo figuras y estilo inline
sns.set()

# URL de dataset's

base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
confirmados_data_url = base_url + 'time_series_covid19_confirmed_global.csv'
muertes_data_url = base_url + 'time_series_covid19_deaths_global.csv'
recuperados_data_url = base_url+ 'time_series_covid19_recovered_global.csv'

# Importando datasets como pandas dataframes
try:
  data_confirmados = pd.read_csv(confirmados_data_url)
  data_muertos = pd.read_csv(muertes_data_url)
  data_recuperados = pd.read_csv(recuperados_data_url)
except HTTPError:
  sys.exit("No se pudieron importar los datasets, verifique las URL")

---
## 2.3 Declaracion de funciones necesarias para el Desarrollo

In [3]:
# Listado de todos los paises del dataset
lista_paises = list(data_confirmados['Country/Region'].drop_duplicates())

# Funciones

# Funcion para agrupar por pais
def agrupar(df):
    # Agrupo por pais y elimino las columnas "Lat" y "Long"
    data = df.groupby(['Country/Region']).sum().drop(['Lat', 'Long'], axis=1)
    # Realizo transposicion del dataset
    data = data.transpose()
    # Seteo los indeces (Fechas) como DateTimeIndex
    datetime_index = pd.DatetimeIndex(data.index)
    data.set_index(datetime_index, inplace=True)
    return data

# Funcion para grafico con respecto al tiempo
def grafico(df, paises, titulo, eje_y):
    if len(paises) == 0:
      return
    if len(paises) == len(lista_paises):
      ax = df.plot(figsize=(30,15), linewidth=2, marker='.', fontsize=20, logy=False)
    else:
      ax = df[paises].plot(figsize=(20,10), linewidth=2, marker='.', fontsize=20, logy=False)
    ax.legend(ncol=5, loc='upper left')
    plt.xlabel("Fecha", fontsize=20);
    plt.ylabel(eje_y, fontsize=20);
    plt.title(titulo, fontsize=20);

# Funcion pronostico
def pronostico(df, paises, Periodo, eje_y):

  if len(paises) == 0:
      return

  df = df.rename(columns={'Country/Region':'Country_Region'})
  df = df[df.Country_Region.isin(paises)]

  values = []
  for date in (df.drop(['Province/State', 'Country_Region', 'Lat', 'Long'], axis=1)):
    values.append(df[date].sum())

  date = list(df.drop(['Province/State', 'Country_Region', 'Lat', 'Long'], axis=1))

  df = pd.DataFrame(list(zip(date,values)), columns = ['ds','y'])

  # Cambiar formato de la fecha de aaaa/mm/dd a aaaa-mm-dd
  df['ds'] = pd.to_datetime(df['ds'],infer_datetime_format=True)
  # Me quedo con los registros posteriores a 2020-05-01 (1 de mayo de 2020)
  df = df[df['ds']>"2020-05-01"]

  # Me quedo solo con y  (casos totales) y ds (fecha) Todo esto para el pais seleccionado
  df = df[['y','ds']]

  # Al asignar estos cambios al mismo df, este queda modificado para posteriores analisis

  # Run Prophet to get predictions
  pred = Prophet(yearly_seasonality=False, daily_seasonality=False)

  # Fit (Entrenar) entrena con los datos de entrada "df"
  pred.fit(df)
  future = pred.make_future_dataframe(periods=Periodo * 30)

  forecast = pred.predict(future)

  #Plot the prediction Graph
  graph = pred.plot(forecast)

  plt.xlabel("Fecha", fontsize=15);
  plt.ylabel(eje_y, fontsize=15);
  plt.title("Pronostico en un peridodo de " + str(Periodo) + " meses", fontsize=15);

---
# 3 Desarrollo

---
## 3.1 Grafico con respecto al tiempo

In [ ]:
paises_seleccionados = []

@interact
def graficar(Pais = ['...']+lista_paises, Mundial = False, Filtro = ['...','Casos confirmados','Muertos','Recuperados']):

  global paises_seleccionados

  if Pais != '...':
    paises_seleccionados.append(Pais)
    paises_seleccionados = list(pd.unique(paises_seleccionados))

  if Mundial:
    paises = lista_paises
  elif Mundial == False:
    paises = paises_seleccionados
    display(paises)

  if Filtro == "Casos confirmados":
    df = agrupar(data_confirmados)
    grafico(df, paises, 'Casos confirmados con respecto al tiempo', 'Casos confirmados')
  elif Filtro == 'Muertos':
    df = agrupar(data_muertos)
    grafico(df, paises, 'Muertos con respecto al tiempo', 'Muertes') 
  elif Filtro == 'Recuperados':
    df = agrupar(data_recuperados)
    grafico(df, paises, 'Recuperados con respecto al tiempo', 'Recuperados') 

---
## 3.2 Pronostico de casos

In [ ]:
pais = []

@interact
def pronosticar(Pais = ['...']+lista_paises, Mundial = False, Filtro = ['...','Casos Confirmados','Muertos','Recuperados'], Periodo = (1,12,1)):
  
  global pais
  if Pais != '...':
    pais = [Pais]

  if Mundial:
    pais = lista_paises
  if Filtro == 'Casos Confirmados':
    pronostico(data_confirmados,pais,Periodo,"Casos Confirmados")
  elif Filtro == 'Muertos':
    pronostico(data_muertos,pais,Periodo,"Muertes")
  elif Filtro == 'Recuperados':
    pronostico(data_recuperados,pais,Periodo,"Recuperados")

---
# 4 Conclusiones

El tema elegido sirve para visualizar la idea de HPC, al utilizar grandes volumenes de datos en los 3 datasets. Se realiza un filtrado de muchos datos para las visualizacion y se utilizan los mismo para poder pronosticar a futuro, lo cual es un campo muy interesante para explorar, ya que tiene mucho aplicacion tanto para el tema Covid como para cualquier otra enfermedad, o datos de otra indole.

---
# 5 Bibliografia

**Codigo de la idea**

https://colab.research.google.com/github/hugobowne/COVID-19-EDA-tutorial/blob/master/notebooks/1-COVID-19-EDA-solution.ipynb#scrollTo=4LdokSt3aIYE

**Documentacion Pandas**

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html

**Documentacion Matplotlib**

https://matplotlib.org

**Prophet**

* **Documentacion**

  https://facebook.github.io/prophet/docs/quick_start.html#python-api

* **Codigo idea**

  https://medium.com/@simonprdhm/prediction-of-the-number-of-covid-19-cases-using-google-colab-python-and-facebooks-prophet-732caee170f8